# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115167e+02     1.668474e+00
 * time: 0.621945858001709
     1     1.044781e+01     8.189736e-01
 * time: 2.141281843185425
     2    -1.152285e+01     9.719055e-01
 * time: 2.741001844406128
     3    -3.422990e+01     7.569619e-01
 * time: 3.6198580265045166
     4    -4.745309e+01     5.159999e-01
 * time: 4.4741480350494385
     5    -5.696734e+01     1.771263e-01
 * time: 5.396565914154053
     6    -5.986522e+01     1.240210e-01
 * time: 5.969390869140625
     7    -6.097332e+01     4.298980e-02
 * time: 6.5471508502960205
     8    -6.140554e+01     6.104528e-02
 * time: 7.133201837539673
     9    -6.168138e+01     3.213154e-02
 * time: 7.729043006896973
    10    -6.190080e+01     2.284852e-02
 * time: 8.311364889144897
    11    -6.204510e+01     1.875989e-02
 * time: 8.888195991516113
    12    -6.210393e+01     1.456123e-02
 * time: 9.485594987869263
    13    -6.215771e+01     9.898902e-03
 * time: 10.057114839553833
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671085
    AtomicLocal         -18.8557719
    AtomicNonlocal      14.8522657
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485391 
    Xc                  -19.3336828

    total               -62.261666457648
